In [1]:
import pypsa
import numpy as np
import pandas as pd
import os
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from rpgm_algo import RpgAlgorithm


In [2]:
class Grid(pypsa.Network):
    def __init__(self, csv_folder_name=None, **kwargs):
        super(Grid, self).__init__(csv_folder_name, **kwargs)

    def __str__(self):
        return "Overriden class."

In [3]:
network = Grid(csv_folder_name="examples/scigrid-de/scigrid-with-load-gen-trafos/")

In [4]:
points = np.array(network.buses[["x", "y"]])

In [ ]:
for idx in np.arange(1, 600, 30):

    # grow small MV network
    # this can be iterated 
    # TODO: determine proper parameters ...
    # TODO: choose buses to connect to ...

    #choose a point and construct random graph around it
    #idx = 5

    print idx, points[idx]

    g = RpgAlgorithm()
    g.debug = False
    g.set_params(n=11, n0=10, r=0, p=0., q=0.)
    # start with random locations and replace first row with points[idx]
    # first node will be connected via a transformer
    connector = 0
    coords = np.zeros([g.n, 2])
    coords[0] = points[idx]
    for i in xrange(1, g.n):
        coords[i] = g._get_coords(sampling="uniform", centre=points[idx], boundaries=[.5, .5])
    g.setup_locations(sampling="data", locations=coords)

    g.initialise()
    g.grow()

    # add g.n new nodes
    # add links between them
    # add transformer between points[idx] and connector

    for j in xrange(g.n):
        network.add("Bus", 
                    "MV{}-{}".format(idx, j),
                    v_nom=10, 
                    x=coords[j, 0], 
                    y=coords[j, 1])
        print "MV{}-{}".format(idx, j)

        network.add("Load",
                "Load MV{}-{}".format(idx, j),
                bus="MV{}-{}".format(idx, j),
                p_set=10)

    for i, edge in enumerate(g.edgelist()):
        network.add("Line", 
                    "MV{}-Line{}".format(idx, i), 
                    x=0.01, 
                    r=0.00001, 
                    s_nom=60,
                    bus0="MV{}-{}".format(idx, edge[0]),
                    bus1="MV{}-{}".format(idx, edge[1]))

    network.add("Transformer", 
                "Transformer MV{}".format(idx), 
                bus0=idx,
                bus1="MV{}-{}".format(idx, connector),
                x=0.1,
                x_pu=0.00005)

In [5]:
network.determine_network_topology()

In [7]:
network.consistency_check()

In [8]:
contingency_factor = 0.7

network.lines.s_nom = contingency_factor*network.lines.s_nom

#There are some infeasibilities without small extensions
for line_name in ["316","527","602"]:
    network.lines.loc[line_name,"s_nom"] = 1200

In [11]:
network.now = network.snapshots[0]

solver_name = "glpk"

network.lopf()

# somehow, the added transformer does not seem to work?


('ok', 'optimal')

In [13]:
colors = np.isnan(network.branches()["length"]).map(lambda v: "b" if v else "orange")
colors = network.lines.voltage.map(lambda v: "c" if v == 220000 else "orange" if v == 380000 else "k")

v_color = np.abs(network.buses_t.p.loc[network.now])

network.plot(bus_sizes=v_color)

(<matplotlib.collections.PathCollection at 0x7f17fc96e0d0>,
 <matplotlib.collections.LineCollection at 0x7f17fc96eb50>)